#ACCESS-S Ensemble Simulation

Run AWRA-L in ensemble mode

In [21]:
from awrams.utils.settings import DB_OPEN_WITH
print(DB_OPEN_WITH)

# This must be set to _fallthrough in order to function correctly
# Modify your config if it is not!

_fallthrough


In [22]:
from awrams.simulation.server import SimulationServer
from awrams.models import awral

from awrams.utils import extents
from awrams.utils import datetools as dt
from awrams.utils import mapping_types as mt

import os

In [23]:
imap = awral.get_default_mapping()
#change_forcing(imap)

### create model output map and add nodes for saving some model outputs to netcdf files

In [24]:
def build_output_mapping(outpath,varlist):
    from awrams.utils.nodegraph import nodes,graph
    from awrams.models.awral.template import DEFAULT_TEMPLATE

    ### populate output map with all model outputs
    output_map = awral.get_output_nodes(DEFAULT_TEMPLATE)

    for v in varlist:
        output_map.mapping[v+'_ncsave'] = nodes.write_to_annual_ncfile(outpath,v)

    return output_map

### initialise states from netcdf files

In [25]:
def initial_states_from_files(mapping,data_path):
    
    if 'mapping' in mapping:
        mapping = mapping['mapping']

    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    mapping['init_sr'] = nodes.init_state_from_ncfile(data_path,'sr_bal*','sr_bal')
    mapping['init_sg'] = nodes.init_state_from_ncfile(data_path,'sg_bal*','sg_bal')

    HRU = {'_hrusr':'_sr','_hrudr':'_dr'}
    for hru in ('_hrusr','_hrudr'):
        for state in ["s0","ss","sd",'mleaf']:
            mapping['init_'+state+hru] = nodes.init_state_from_ncfile(data_path,state+HRU[hru]+'*',state+HRU[hru])


In [26]:
states_path = '/data/cwd_awra_data/awra_test_outputs/Scheduled_v5_awraprod1/states/'

In [27]:
initial_states_from_files(imap,states_path)
#sim = Server(awral)
#sim.run(imap,omap,period,extent)

In [28]:
BASE_ACCESS_INPUT_PATH = '/data/cwd_awra_data/awra_inputs/ACCESS/'

In [29]:
ACCESS_FORCING_VARS = ['pr','rsds','tasmin','tasmax']
AWRAL_ACCESS_FORCING_MAP = {'tmax': 'tasmax', 'tmin': 'tasmin', 'precip': 'pr', 'solar': 'rsds'}
N_ACCESS_ENSEMBLE = 11

In [30]:
ref_file_access =  os.path.join(BASE_ACCESS_INPUT_PATH,'tasmin/da5_tasmin_20060101_e01.nc')
e_access = extents.Extent.from_file(ref_file_access)

e_all = extents.get_default_extent()
eat = e_access.translate(e_all.parent_ref)
eat = eat.factory.get_by_boundary_coords(*e_all.bounding_rect())

In [31]:
# Approximate bounding rect covers all of NSW

e_sw_bounds = eat.factory.get_by_boundary_coords(-28,-38,140,154)
e_sw_bounds.cell_count

47175

In [32]:
def get_access_ens_filename(var,datestr,ens_number):
    e_str = 'e'+str(ens_number).zfill(2)
    return('da5_%s_%s_%s.nc' % (var,datestr,e_str))

In [33]:
get_access_ens_filename('tasmin','20060101',1)

'da5_tasmin_20060101_e01.nc'

In [34]:
def set_forcing_ensemble(imap, datestr, ensemble):
    from awrams.utils.nodegraph.nodes import forcing_from_ncfiles
    from os.path import join
    from os import getcwd
    
    for k,v in AWRAL_ACCESS_FORCING_MAP.items():
        fname = get_access_ens_filename(v,datestr,ensemble)
        vdir = os.path.join(BASE_ACCESS_INPUT_PATH,v)
        imap.mapping[k+'_f'] = forcing_from_ncfiles(vdir,fname,v)


In [35]:
FORECAST_START = dt.day('1 jan 2006')
date_str = FORECAST_START.strftime('%Y%m%d')
date_str

'20060101'

In [36]:
from awrams.simulation.server import SimulationServer

In [37]:
server = SimulationServer(awral)

In [38]:
server.num_workers = 8
server.read_ahead = 4

In [39]:
BASE_OUTPUT_PATH = '/data/cwd_awra_data/awra_test_outputs/SydneyWater/'

In [40]:
assert(False)

for month in ['october']:
    for year in range(2008,2012):
        FORECAST_START = dt.day('1 %s %s' % (month, year))
        date_str = FORECAST_START.strftime('%Y%m%d')
        print('Running %s' % date_str)
        for ens_n in range(1,N_ACCESS_ENSEMBLE+1):
            print('Running for ensemble %s' %ens_n)
            out_path = os.path.join(BASE_OUTPUT_PATH, 'fcast%s' % date_str, 'e%s' % str(ens_n).zfill(2))
            omap = build_output_mapping(out_path,['s0','ss','sd','e0','etot','qtot'])
            set_forcing_ensemble(imap,date_str,ens_n)
            period = dt.dates(FORECAST_START,FORECAST_START+200)
            server.run(imap,omap,period,e_sw_bounds)

2017-08-21 01:37:15,352 INFO Getting I/O dataspecs...
2017-08-21 01:37:46,277 INFO Initialising output files...
2017-08-21 01:37:46,593 INFO Building buffers...
2017-08-21 01:37:52,567 INFO Running simulation...


Running 20081001
Running for ensemble 1
worker pid: 31676
worker pid: 31679
worker pid: 31682
worker pid: 31697
worker pid: 31691


2017-08-21 01:38:34,312 INFO completed 5.36%
2017-08-21 01:37:52,705 INFO writer pid: 31717
2017-08-21 01:37:52,964 INFO reader pid: 31716
2017-08-21 01:38:51,555 INFO completed 10.71%
2017-08-21 01:38:58,875 INFO completed 16.07%
2017-08-21 01:39:25,689 INFO completed 21.43%
2017-08-21 01:39:37,897 INFO completed 26.79%
2017-08-21 01:39:46,883 INFO completed 32.14%
2017-08-21 01:40:09,214 INFO completed 37.50%
2017-08-21 01:40:25,174 INFO completed 42.86%
2017-08-21 01:40:31,976 INFO completed 48.21%
2017-08-21 01:40:52,709 INFO completed 53.57%
2017-08-21 01:41:05,763 INFO completed 58.93%
2017-08-21 01:41:20,895 INFO completed 64.29%
2017-08-21 01:41:32,637 INFO completed 69.64%
2017-08-21 01:41:44,228 INFO completed 75.00%
2017-08-21 01:42:00,165 INFO completed 80.36%
2017-08-21 01:42:12,133 INFO completed 85.71%
2017-08-21 01:42:15,145 INFO completed 91.07%
2017-08-21 01:42:19,318 INFO completed 96.43%
2017-08-21 01:42:21,394 INFO Completed period October - December 2008 - 1 of 2


Running for ensemble 2
reader pid: 31716
worker pid: 31703
worker pid: 31712
worker pid: 31709
worker pid: 31896
worker pid: 31897
worker pid: 31902
worker pid: 31911
worker pid: 31917
worker pid: 31923


2017-08-21 01:44:24,287 INFO completed 5.36%
2017-08-21 01:43:52,643 INFO writer pid: 31943
2017-08-21 01:43:52,649 INFO reader pid: 31939
2017-08-21 01:44:33,265 INFO completed 10.71%
2017-08-21 01:44:39,518 INFO completed 16.07%
2017-08-21 01:44:51,788 INFO completed 21.43%
2017-08-21 01:44:59,718 INFO completed 26.79%
2017-08-21 01:45:06,104 INFO completed 32.14%
2017-08-21 01:45:20,100 INFO completed 37.50%
2017-08-21 01:45:29,527 INFO completed 42.86%
2017-08-21 01:45:34,287 INFO completed 48.21%
2017-08-21 01:45:46,480 INFO completed 53.57%
2017-08-21 01:45:57,596 INFO completed 58.93%
2017-08-21 01:46:04,093 INFO completed 64.29%
2017-08-21 01:46:12,918 INFO completed 69.64%
2017-08-21 01:46:22,396 INFO completed 75.00%
2017-08-21 01:46:32,928 INFO completed 80.36%
2017-08-21 01:46:43,714 INFO completed 85.71%
2017-08-21 01:46:46,599 INFO completed 91.07%
2017-08-21 01:46:48,275 INFO completed 96.43%
2017-08-21 01:46:50,889 INFO Completed period October - December 2008 - 1 of 2


...

Running for ensemble 10
worker pid: 7397
reader pid: 7407
worker pid: 7521
worker pid: 7522
worker pid: 7530
worker pid: 7536
worker pid: 7542
worker pid: 7555
worker pid: 7548
worker pid: 7560


2017-08-21 03:05:42,614 INFO completed 5.36%
2017-08-21 03:05:24,058 INFO writer pid: 7571
2017-08-21 03:05:24,081 INFO reader pid: 7570
2017-08-21 03:05:44,130 INFO completed 10.71%
2017-08-21 03:05:55,305 INFO completed 16.07%
2017-08-21 03:05:59,984 INFO completed 21.43%
2017-08-21 03:06:00,902 INFO completed 26.79%
2017-08-21 03:06:01,609 INFO completed 32.14%
2017-08-21 03:06:03,028 INFO completed 37.50%
2017-08-21 03:06:03,736 INFO completed 42.86%
2017-08-21 03:06:04,416 INFO completed 48.21%
2017-08-21 03:06:06,150 INFO completed 53.57%
2017-08-21 03:06:07,318 INFO completed 58.93%
2017-08-21 03:06:08,458 INFO completed 64.29%
2017-08-21 03:06:09,128 INFO completed 69.64%
2017-08-21 03:06:10,229 INFO completed 75.00%
2017-08-21 03:06:11,267 INFO completed 80.36%
2017-08-21 03:06:12,326 INFO completed 85.71%
2017-08-21 03:06:12,786 INFO completed 91.07%
2017-08-21 03:06:13,442 INFO completed 96.43%
2017-08-21 03:06:13,898 INFO Completed period October - December 2011 - 1 of 2
20

Running for ensemble 11
reader pid: 7570
worker pid: 7684
worker pid: 7685
worker pid: 7688
worker pid: 7699
worker pid: 7705
worker pid: 7713
worker pid: 7720
worker pid: 7723


2017-08-21 03:07:27,297 INFO completed 5.36%
2017-08-21 03:07:17,617 INFO writer pid: 7734
2017-08-21 03:07:17,630 INFO reader pid: 7733
2017-08-21 03:07:28,022 INFO completed 10.71%
2017-08-21 03:07:28,776 INFO completed 16.07%
2017-08-21 03:07:29,783 INFO completed 21.43%
2017-08-21 03:07:30,702 INFO completed 26.79%
2017-08-21 03:07:31,422 INFO completed 32.14%
2017-08-21 03:07:32,564 INFO completed 37.50%
2017-08-21 03:07:33,561 INFO completed 42.86%
2017-08-21 03:07:34,209 INFO completed 48.21%
2017-08-21 03:07:35,435 INFO completed 53.57%
2017-08-21 03:07:36,519 INFO completed 58.93%
2017-08-21 03:07:37,392 INFO completed 64.29%
2017-08-21 03:07:38,300 INFO completed 69.64%
2017-08-21 03:07:39,186 INFO completed 75.00%
2017-08-21 03:07:40,193 INFO completed 80.36%
2017-08-21 03:07:41,125 INFO completed 85.71%
2017-08-21 03:07:41,634 INFO completed 91.07%
2017-08-21 03:07:42,289 INFO completed 96.43%
2017-08-21 03:07:42,712 INFO Completed period October - December 2011 - 1 of 2
20

reader pid: 7733
